In [14]:
# !pip install sklearn --upgrade

In [15]:
# install joblib. This will be used to save your model. 
# Restart your kernel after installing 
# !pip install joblib

In [28]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import load_model
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier

# Read the CSV and Perform Basic Data Cleaning

In [33]:
pd.options.display.max_columns = 50
df = pd.read_csv("exoplanet_data.csv")
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,koi_time0bk_err2,koi_impact,koi_impact_err1,koi_impact_err2,koi_duration,koi_duration_err1,koi_duration_err2,koi_depth,koi_depth_err1,koi_depth_err2,koi_prad,koi_prad_err1,koi_prad_err2,koi_teq,koi_insol,koi_insol_err1,koi_insol_err2,koi_model_snr,koi_tce_plnt_num,koi_steff,koi_steff_err1,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,-0.003520,0.586,0.059,-0.443,4.50700,0.11600,-0.11600,874.8,35.5,-35.5,2.83,0.32,-0.19,443,9.11,2.87,-1.62,25.8,2,5455,81,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,-0.000581,0.969,5.126,-0.077,1.78220,0.03410,-0.03410,10829.0,171.0,-171.0,14.60,3.92,-1.31,638,39.30,31.04,-10.49,76.3,1,5853,158,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,-0.000115,1.276,0.115,-0.092,2.40641,0.00537,-0.00537,8079.2,12.8,-12.8,33.46,8.50,-2.83,1395,891.96,668.95,-230.35,505.6,1,5805,157,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,-0.001130,0.701,0.235,-0.478,1.65450,0.04200,-0.04200,603.3,16.9,-16.9,2.75,0.88,-0.35,1406,926.16,874.33,-314.24,40.9,1,6031,169,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,-0.001900,0.762,0.139,-0.532,3.14020,0.06730,-0.06730,686.0,18.7,-18.7,2.77,0.90,-0.30,1160,427.65,420.33,-136.70,40.2,2,6046,189,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


# Select your features (columns)

In [30]:
# To Calculate feature importance

rf = RandomForestClassifier(n_estimators=200)
data = df.drop('koi_disposition',axis=1)
target = df[['koi_disposition']]
rf = rf.fit(data, np.ravel(target))
# rf.score
importances = rf.feature_importances_
sorted(zip(importances, data.columns), reverse=True)

[(0.10901529602111704, 'koi_fpflag_co'),
 (0.10057417016428973, 'koi_fpflag_nt'),
 (0.07248615774426603, 'koi_fpflag_ss'),
 (0.06053697901255485, 'koi_model_snr'),
 (0.05090534337133842, 'koi_prad'),
 (0.03874867375290441, 'koi_duration_err2'),
 (0.03652078456317114, 'koi_fpflag_ec'),
 (0.03402882236346971, 'koi_steff_err1'),
 (0.031898711875632674, 'koi_prad_err1'),
 (0.02826864304796836, 'koi_prad_err2'),
 (0.027337844265885924, 'koi_duration_err1'),
 (0.024712912775223903, 'koi_time0bk_err2'),
 (0.023880854966981547, 'koi_period'),
 (0.023871066770983343, 'koi_time0bk_err1'),
 (0.023666509710050847, 'koi_steff_err2'),
 (0.022327802495543392, 'koi_duration'),
 (0.01886174746869928, 'koi_impact'),
 (0.0174212212786206, 'koi_depth'),
 (0.016929848211445334, 'koi_insol'),
 (0.01673918549857369, 'koi_insol_err1'),
 (0.01557113274208441, 'koi_period_err2'),
 (0.014921627543241809, 'koi_insol_err2'),
 (0.014875749785938452, 'koi_period_err1'),
 (0.014395504900517657, 'koi_teq'),
 (0.013896

In [31]:
# Set features. This will also be used as your x values.
selected_features = df[[ 'koi_fpflag_co', 'koi_fpflag_nt', 'koi_fpflag_ss', 'koi_model_snr', 'koi_prad','koi_duration_err2',
                        'koi_fpflag_ec', 'koi_steff_err1', 'koi_prad_err1']]
selected_features.head()

,koi_fpflag_co,koi_fpflag_nt,koi_fpflag_ss,koi_model_snr,koi_prad,koi_duration_err2,koi_fpflag_ec,koi_steff_err1,koi_prad_err1
0,0,0,0,25.8,2.83,-0.11600,0,81,0.32
1,0,0,1,76.3,14.60,-0.03410,0,158,3.92
2,0,0,1,505.6,33.46,-0.00537,0,157,8.50
3,0,0,0,40.9,2.75,-0.04200,0,169,0.88
4,0,0,0,40.2,2.77,-0.06730,0,189,0.90


# Create a Train Test Split

Use `koi_disposition` for the y values

In [46]:
y = df['koi_disposition']

In [44]:
X_train, X_test, y_train, y_test = train_test_split(selected_features, y, random_state = 1, stratify=y)
X_train.head()

,koi_fpflag_co,koi_fpflag_nt,koi_fpflag_ss,koi_model_snr,koi_prad,koi_duration_err2,koi_fpflag_ec,koi_steff_err1,koi_prad_err1
4002,1,0,0,12.6,576.14,-0.1790,0,145,30.44
4246,0,0,1,303.8,6.77,-0.0156,0,146,0.51
548,1,0,1,220.3,38.93,-0.0235,0,158,10.98
3953,0,0,1,535.1,33.24,-0.0124,0,122,5.85
2362,0,0,0,134.8,2.26,-0.0343,0,85,0.12


# Pre-processing

Scale the data using the MinMaxScaler and perform some feature selection

In [45]:
# Scale your data
X_scaler = MinMaxScaler()
X_scaler.fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

# Train the Model



In [ ]:

print(f"Training Data Score: {model2.score(X_train_scaled, y_train)}")
print(f"Testing Data Score: {model2.score(X_test_scaled, y_test)}")

# Hyperparameter Tuning

Use `GridSearchCV` to tune the model's parameters

In [ ]:
# Create the GridSearchCV model

In [ ]:
# Train the model with GridSearch

In [ ]:
print(grid2.best_params_)
print(grid2.best_score_)

# Save the Model

In [ ]:
# save your model by updating "your_name" with your name
# and "your_model" with your model variable
# be sure to turn this in to BCS
# if joblib fails to import, try running the command to install in terminal/git-bash
import joblib
filename = 'your_name.sav'
joblib.dump(your_model, filename)